In [1]:
import time
import copy
import random

import torch
import torch.nn as nn

import torchvision
import numpy as np

from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

from pytorch_lightning import Trainer
from pytorch_lightning.core.lightning import LightningModule

In [2]:
epochs = 10
batch = 16
lr = 0.001
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_dataset = torchvision.datasets.MNIST('./mnist', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST('./mnist', train=False, download=True,transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, num_workers=4, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, num_workers=4, shuffle=True)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)
        
        self.act = nn.ReLU()
        self.pool = nn.AdaptiveMaxPool2d(output_size=(1, 1))
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.act(x)
        
        x = self.conv2(x)
        x = self.act(x)
        
        x = self.pool(x)
        x = torch.flatten(x, 1)
        
        x = self.fc1(x)
        x = self.act(x)
        
        x = self.fc2(x)
        return x

In [5]:
m = Net()

In [6]:
class LitNet(LightningModule):
    def __init__(self, model, optimizer, criteria):
        super().__init__()
        self.model = model
        self.optimizer = optimizer
        self.criteria = criteria
    
    def forward(self, x):
        x = self.model(x)
        
    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        
        out = self.model(imgs)        
        loss = self.criteria(out, labels)
        return loss
    
    def validationstep(self, batch, batch_idx):
        imgs, labels = batch
        
        out = self.model(imgs)        
        out = torch.argmax(out, dim=1)
        
        accuracy += accuracy_score(labels.numpy(), out.detach().numpy())
        f1 += f1_score(labels.numpy(), out.detach().numpy(), average='macro')
        
        return {'accuracy': accuracy, 'f1': f1}
    
    def configure_optimizers(self):
        return self.optimizer

In [13]:
model_wrapper = LitNet(m, torch.optim.Adam(m.parameters(), lr=lr), torch.nn.CrossEntropyLoss())
# model
trainer = Trainer()
trainer.fit(model_wrapper, train_dataloader)

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name     | Type             | Params
----------------------------------------------
0 | model    | Net              | 56.9 K
1 | criteria | CrossEntropyLoss | 0     
----------------------------------------------
56.9 K    Trainable params
0         Non-trainable params
56.9 K    Total params
0.228     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

1